In [11]:
import json
import ast

In [55]:
master = {}
master['apps'] = []

In [56]:
directory = '../../data/cache/2020-05-02_21-41-54'


In [57]:
import os

for file in os.listdir(directory):
    if file.endswith(".txt"):
        cache_file = os.path.join(directory, file)

        with open(cache_file, encoding='utf8') as f:
            for line in f:
                app_details = ast.literal_eval(line)
                master['apps'].append(app_details)

In [59]:
with open(directory+'/steamstore.json','w') as out:
    json.dump(master,out)